In [1]:
file_name='input/md/english/FARMER PRODUCER ORGANISATIONS.md'
md_file=open(file_name,'r',encoding='utf-8')
md=md_file.read()
chunks=md.split('\n')

# chunks=[chunk for chunk in chunks if chunk!='']
# for chunk in chunks:
#     print(chunk,end='\n*****\n')

In [2]:
chunks

['',
 '## Nabard Farmer Producer Organisations Frequently Asked Questions (Faqs)',
 '',
 '# National Bank For Agriculture Mumbai 2015',
 '',
 '"The issue of profitability of small holding based agriculture',
 'has assumed importance in view of increasing proportion of',
 'small and marginal farmers in the country. I propose to',
 "supplement NABARD's Producers' organization development",
 "fund for Producer's development and upliftment called",
 'PRODUCE with a sum of Rs 200 crore which will be utilized',
 'for building 2,000 producers organizations across the country',
 'over the next two years."',
 '',
 "Shri Arun Jaitley, Hon'ble Union Finance Minister, Govt. of India",
 '',
 '# And Rural Development',
 '',
 "## Nabard'S Mission",
 '',
 '  Promote sustainable and equitable ',
 'agriculture and rural prosperity through ',
 'effective credit support, related services, ',
 '  institution development and other ',
 '         innovative initiatives.',
 '',
 '# Farmer Producer Organisation

In [5]:
def table_parser(chunks,index):
    inside_table = False
    current_table = ''

    for i,chunk in enumerate(chunks):
        if inside_table:
            # Check if the current chunk is the end of the tablere
            if chunk.startswith('|'):
                inside_table = True
                current_table=current_table+'\n'+chunk
            else:
                
                return current_table,(index+i)
        elif chunk.startswith('|'):
            inside_table = True
            current_table=chunk
    #     elif chunk=='':
    #         continue
#         else:
#             print('post',(index+i))
#             return combined_chunks,(index+i)
    return 0

    
def para_parser(chunks,index):
    inside_para=False
    current_para=''
    temp=['#','## ','### ','#### ','##### ','###### ','-','|']
    for i,chunk in enumerate(chunks):
        if chunk=='':
            i=i+1            
        elif inside_para:
            if chunk[0] not in temp:
                inside_para=True
                current_para=current_para+'\n'+chunk
            else:
                return (current_para,(index+i))            
        elif chunk[0] not in temp:
            inside_para=True
            current_para=chunk
            
    return (current_para,(index+i))

def list_parser(chunks,index):
    inside_list=False
    current_list=''
    temp=['#','## ','### ','#### ','##### ','###### ','-','|']
    for i,chunk in enumerate(chunks):
        if inside_list:
            if chunk=='' or chunk.startswith('- ') or chunk.startswith('* '):
                return (current_list,(index+i))
            if chunk[0] not in temp:
                current_list=current_list+'\n'+chunk
        elif chunk.startswith('-') or chunk.startswith('* '):
            current_list=chunk
            inside_list=True
            
    return (current_para,(index+i))   

In [6]:
# for i in range(len(chunks)):
i=0
combined_chunks=[]
temp=['#','## ','### ','#### ','##### ','###### ','-','*','|']
while i<len(chunks):
    
    if chunks[i]=='':
        combined_chunks.append({'type':'return','content':chunks[i]})
        i=i+1
    elif chunks[i].startswith('|'):
        chunk,i=table_parser(chunks[i:],i)
        combined_chunks.append({'type':'table','content':chunk})
        
    elif chunks[i].startswith('# ') or chunks[i].startswith('## ') or chunks[i].startswith('### ') or chunks[i].startswith('#### ') or chunks[i].startswith('##### '):
        combined_chunks.append({'type':'heading','content':chunks[i]})
        i=i+1
    
    elif chunks[i].startswith('- ') or chunks[i].startswith('* '):
        chunk,i=para_parser(chunks[i:],i)
        combined_chunks.append({'type':'list','content':chunk})
    
    elif chunks[i][0] not in temp and (i+1)!=len(chunks):
        # chunk,i=para_parser(chunks[i:],i)
        chunk=chunks[i].strip()
        combined_chunks.append({'type':'para','content':chunk})
        i=i+1
    else:
        i=i+1

In [19]:
a='     '.strip()
print(len(a))



0


In [7]:
combined_chunks = [chunk for chunk in combined_chunks if chunk['type'] != 'return']
# combined_chunks

In [ ]:
inside_pair=False
paired_chunks=[]
current_pair={'type':'heading-para pair','content':{}}
for chunk in combined_chunks:
    if chunk['type']=='heading':
        inside_pair=True
        current_pair['content']['heading']=chunk['content']
    elif chunk['type']=='para':
        if inside_pair:
            current_pair['content']['para']=chunk['content']
            paired_chunks.append(current_pair)
            inside_pair=False
            current_pair={'type':'heading-para pair','content':{}}
        else:
            paired_chunks.append(chunk)
    else:
        paired_chunks.append(chunk)

In [ ]:
# def count_level(content):
#     level
#     if content.startswith('# '):
#         level=1
#     elif content.startswith('## '):
#         level=2
#     elif content.startswith('### '):
#         level=3
#     elif content.startswith('#### '):
#         level=4
#     elif content.startswith('##### '):
#         level=5
#     elif content.startswith('###### '):
#         level=6
    

#     return level

In [8]:
def assign_ids_and_parents(chunks):
    id_counter = {'1': 0, '2': 0, '3': 0,'4':0,'5':0} 
    parent_stack = [] 
    
    for chunk in chunks:
        if chunk['type'] == 'heading':
            level = chunk['content'].count('#')
#             level=count_level(chunk['content'])
#             print('hehe',level)
            id_counter[str(level)] += 1
            chunk['id'] = f'{level}{id_counter[str(level)]}'
            
            if parent_stack==[] or level==1:
                parent_stack = [chunk['id']]
            else:
                buff=int(str(parent_stack[0])[0])-1
                while len(parent_stack)>=(level-buff):
                    parent_stack.pop()
                parent_stack.append(chunk['id'])
            chunk['parents'] = parent_stack[:-1]
#             print(parent_stack)
            
        else:
            chunk['parents'] = parent_stack.copy() if parent_stack else []
    
    return chunks
final_chunks=assign_ids_and_parents(combined_chunks)
final_chunks

[{'type': 'heading',
  'content': '## Nabard Farmer Producer Organisations Frequently Asked Questions (Faqs)',
  'id': '21',
  'parents': []},
 {'type': 'heading',
  'content': '# National Bank For Agriculture Mumbai 2015',
  'id': '11',
  'parents': []},
 {'type': 'para',
  'content': '"The issue of profitability of small holding based agriculture\nhas assumed importance in view of increasing proportion of\nsmall and marginal farmers in the country. I propose to\nsupplement NABARD\'s Producers\' organization development\nfund for Producer\'s development and upliftment called\nPRODUCE with a sum of Rs 200 crore which will be utilized\nfor building 2,000 producers organizations across the country\nover the next two years."\nShri Arun Jaitley, Hon\'ble Union Finance Minister, Govt. of India',
  'parents': ['11']},
 {'type': 'heading',
  'content': '# And Rural Development',
  'id': '12',
  'parents': []},
 {'type': 'heading',
  'content': "## Nabard'S Mission",
  'id': '22',
  'parents':

In [12]:
final_chunks[384]

{'type': 'para', 'content': ' ', 'parents': ['14', '2184']}

In [81]:
for chunk in final_chunks:
    if chunk['type']=='heading':
        print(chunk['id'],chunk['content'])

11 # Contributing Guidlines
21 ## ⚙Setup Git If You've Never Used Git Before.
22 ## ⭐Contributing
31 ### Step 0: Find an issue
32 ### Step 1 : Fork the Project
33 ### Step 2: Branch
34 ### Step 3: Work on the issue assigned
35 ### Step 4: Commit
36 ### Step 5: Work Remotely
37 ### Step 6: Pull Request
38 ### All the best! 🥇
23 ## 🛡️Code of Conduct
39 ### Our Pledge
310 ### Our Standards
311 ### Our Responsibilities
312 ### Scope
313 ### Enforcement
314 ### Attribution


In [71]:
len(final_chunks)

25

In [59]:
for chunk in final_chunks:
    for key,value in chunk.items():
        print(f'{key}-{value}\n')
    print('\n\n')

type-return

content-

parents-[]




type-heading

content-## रबी फसलों को सघन पद्धतियाँ 209

id-21

parents-[]




type-return

content-

parents-['21']




type-para

content-कृषि विभाग, उत्त्तर प्रदेश
लखनऊ

parents-['21']




type-table

content-|॥2

parents-['21']




type-para

content-विधानभवन, लखनऊ
है॥
(का5शॉट
सूर्य प्रताप शाही
मंत्री,
कृषि, कृषि शिक्षा, कृषि अनुसंधान
सन्देश
हा ६83 प्रदेश
ढ
कक
यह हर्ष का विषय है कि कृषि विभाग, उत्त्तर प्रदेश "रबी फसलों की सघन पद्धतियाँ" नामक पुस्तक का
निरन्तर प्रकाशन कर रहा है। यह पुस्तक कृषि उत्पादन में संलग्न कृषक बन्धुओं के तकनीकी ज्ञान में
।है अभिवृद्धि के दृष्टिकोण सेअत्यन्त उपयोगीसिद्ध होगी |
ि
३
कृषक हमारे देश की अर्थव्यवस्था की धुरी हैं |इनके विकास के बिना देश के विकास की अवधारणा एक
।

parents-['21']




type-table

content-|

parents-['21']




type-para

content-दिवास्वप्न जैसा है। इस बात को ध्यान में रखते हुए "रबी फसलों की सघन पद्धतियाँ-209" कृषकों के डे
समक्ष प्रस्तुत
कीजा रही है। फसलों के उत्पादन एवं उत्पादकता हेतु मृदा उर्वरता के 

## DUMP

In [ ]:
combined_chunks = []
inside_table = False
current_table = ''

for chunk in chunks:
    if inside_table:
        # Check if the current chunk is the end of the table
        if chunk.startswith('|'):
            inside_table = True
            current_table=current_table+'\n'+chunk
        else:
            combined_chunks.append(current_table)
            combined_chunks.append(chunk)
            inside_table=False
    elif chunk.startswith('|'):
        inside_table = True
        current_table=chunk
#     elif chunk=='':
#         continue
    else:
        combined_chunks.append(chunk)

In [ ]:
inside_pair=False
current_pair=''
for chunk in combined_chunks:
    if chunk['type']=='heading':
        iside_pair=True
    elif chunk['type']=='para':
        if inside_pair

In [ ]:
for i in combined_chunks:
    print(i,end='\n************\n')

## GPT Stuff (No use)

In [ ]:
from markdown_it import MarkdownIt
from bs4 import BeautifulSoup

In [ ]:
file='Value_chain_financing.md'
file=open(file,'r',encoding='utf-8')
md_text=file.read()

In [ ]:
from markdown_it import MarkdownIt

def md_to_html(md_file_path, html_file_path):
    # Read Markdown content from file
    with open(md_file_path, 'r', encoding='utf-8') as md_file:
        md_text = md_file.read()
    md = MarkdownIt()
    html_content = md.render(md_text)
    with open(html_file_path, 'w', encoding='utf-8') as html_file:
        html_file.write(html_content)
        
md_path = "Value_chain_financing.md"
html_path = "output.html"
md_to_html(md_path, html_path)


In [ ]:
file=open('output.html', 'r', encoding='utf-8')
html_content = file.read()
page_data = {}
current_main_heading = ''
current_subheading = ''
crop_soup = BeautifulSoup(html_content, 'html.parser')
# print(crop_soup)
tags = crop_soup.find_all(['h4','h1','h2','h3', 'p', 'li','table'])

for tag in tags:
    if tag.name == 'h4':
        current_main_heading = tag.text.strip()
        current_subheading = ''  # Reset subheading when a new main heading is found
        if current_main_heading not in page_data:
            page_data[current_main_heading] = {}
    elif tag.name == 'p' and tag.find('strong'):
        current_subheading = tag.text.strip()
        if current_main_heading not in page_data:
            page_data[current_main_heading] = {}
        if current_subheading not in page_data[current_main_heading]:
            page_data[current_main_heading][current_subheading] = []
    elif current_main_heading and current_subheading:
        # Only add content if both main heading and subheading are available
        if tag.name=='table':
            data=table_parser(tag)
        else:
            data=tag.text.strip()
        page_data[current_main_heading][current_subheading].append(data)


    else:
        if current_main_heading and not current_subheading:
            current_subheading='General'
            page_data[current_main_heading][current_subheading]=[]
        if not current_main_heading and not current_subheading:
            current_main_heading='General'
            current_subheading='General'
            page_data[current_main_heading]={}
            page_data[current_main_heading][current_subheading]=[]
        if tag.name=='table':
            data=table_parser(tag)
        else:
            data=tag.text.strip()
        page_data[current_main_heading][current_subheading].append(data)


In [ ]:
page_data

In [ ]:
md = MarkdownIt()
tokens = md.parse(md_text)

headers = []
current_header = None

for token in tokens:
    print(token,end='\n\n\n')